# Get edge parameterization for differnt MC /data productions

1. Load production lists - filtered data from alien
2. Extract skimmed data - create pandas with subset of information
3. Make parameteratization of performance (as in  PWGPP-567/comparisondEdgePerf.ipynb)
* Variables to describe
  * Ncrossed raw and Ncl
  * Fraction of associated clusters
  * DCA resolution+bias
  * dEdx bias
  * TPC/ITS chi2
  
*  Variables binning resp. for ML:
  * local position (at radii)
  * 1/dEdx
  * multiplicity
  * q/pt
  

4.Visulaize/compare - regression maps  

In [ ]:
import ROOT
from RootInteractive.Tools.aliTreePlayer import *
from RootInteractive.InteractiveDrawing.bokeh.bokehDrawSA import *
from bokeh.models import *
from bokeh.io import output_notebook
from RootInteractive.MLpipeline.NDFunctionInterface import  DataContainer, Fitter
output_notebook()
c  = ROOT.TCanvas("canvas","canvas")

In [ ]:
ROOT.TGrid.Connect("alien","miranov")

# Prepare production lists
* MC new production - LHC19f5b_3
* MC old production - LHC19f5b_2
* MC old production - LHC19f5b

In [ ]:
ROOT.gSystem.Exec("alien_find /alice/sim/2019/LHC19f5b_3/296433/SpacePointCalibrationMerge Filter*root  | sed s_/alice_alien:///alice_> filtered_LHC19f5b_3.list")
ROOT.gSystem.Exec("alien_find /alice/sim/2019/LHC19f5b_2/296433/SpacePointCalibrationMerge Filter*root  | sed s_/alice_alien:///alice_> filtered_LHC19f5b_2.list")
ROOT.gSystem.Exec("alien_find /alice/sim/2019/LHC19f5b/296433/SpacePointCalibrationMerge Filter*root  | sed s_/alice_alien:///alice_> filtered_LHC19f5b.list")
ROOT.gSystem.GetFromPipe("wc filtered*.list")

In [ ]:
#tree = f.Get("highPt")
tree = ROOT.AliXRDPROOFtoolkit.MakeChainRandom("filtered_LHC19f5b_3.list","highPt","",1,0)
tree.SetMarkerStyle(21); tree.SetMarkerSize(0.5);
ROOT.AliDrawStyle.SetDefaults();
ROOT.AliDrawStyle.ApplyStyle("figTemplate");
tree.SetCacheSize(200000000)

In [ ]:
ROOT.AliAnalysisTaskFilteredTree.SetDefaultAliasesHighPt(tree)
#tree.GetListOfAliases().ls()

In [ ]:
ROOT.gROOT.LoadMacro("/eos/user/m/mivanov/github/RootInteractiveTest/JIRA/Tools/rdataframeFilter.C") 

In [ ]:
ROOT.makeRDFrameSnapshot0(tree,"test.root",5)

In [ ]:
f = ROOT.TFile.Open("test.root")
x= f.Get("tree")

In [ ]:
x.Draw("cr40_65:pos40_65>>his40_65","abs(qPt)<1","prof")
c.Draw()

In [ ]:
x.Show(0)

In [ ]:
df=tree2Panda(x,[".*cr.*",".*qP.*",".*pos.*",".*tgl.*"],"qP!=0")
df.head(5)

In [ ]:
from RootInteractive.MLpipeline.NDFunctionInterface import  DataContainer, Fitter
varList=["cr0_25","cr20_45", "cr40_65"]
paramList=["qPt","pos0_25"]
npoints=df.shape[0]
#fitter.Fit()

for varFit in varList:
    dataContainer = DataContainer(df,  paramList, varFit, [npoints//2,npoints//2]) 
    fitter = Fitter(dataContainer)
    fitter.Register_Method('RF10','RandomForest', 'Regressor', n_estimators=100, max_depth=10)
    fitter.Register_Method('RF15','RandomForest', 'Regressor', n_estimators=100, max_depth=15)
    fitter.Fit()
    fitter.AppendStatPandas("RF10",df,varFit)
    fitter.AppendStatPandas("RF15",df,varFit)


In [ ]:
df.head(5)

In [ ]:
tooltips=[("qP","@qP"), ("pz/pt","@tgl"), ("cr0_25","@cr025")]
figureArray= [
    #[['pos0_25'], ['cr0_25RF10Mean'], {"size": 4, "colorZvar":"qPt"}],
    #[['pos20_45'], ['cr20_45RF10Mean'], {"size": 4, "colorZvar":"qPt"}],
    [['pos0_25'], ['cr0_25RF15Mean'], {"size": 4, "colorZvar":"qPt"}],
    [['pos20_45'], ['cr20_45RF15Mean'], {"size": 4, "colorZvar":"qPt"}],
    [['pos0_25'], ['cr0_25RF15Mean-cr0_25RF10Mean'], {"size": 4, "colorZvar":"qPt"}],
    [['pos20_45'], ['cr20_45RF15Mean-cr20_45RF15Mean'], {"size": 4, "colorZvar":"qPt"}],
    ['table']
]
figureLayout=[
    [0,1, {'commonX':0,'y_visible':2,'plot_height':250}],
    [2,3, {'commonX':0,'plot_height':250}],
    [4,{'plot_height':20}],
    {'plot_height':100,'sizing_mode':'scale_width'}
]
widgetArray=[
    ['range', ['qPt']],
    ['range', ['tgl']],
    ['range', ['pos0_25']],
    ['range', ['pos20_45']],
]
widgetLayout=[ [0,1], [2,3], {'sizing_mode':'scale_width'} ]

output_file("makePanda.html")
bokehDrawSA.fromArray(df.head(1000), "qP>0&cr0_25RF15RMS<2", figureArray, widgetArray, layout=figureLayout, tooltips=tooltips,widgetLayout=widgetLayout,sizing_mode="scale_width")

# Backup

In [ ]:
tree.Draw("esdTrack.GetTPCClusterInfo(3,1):sign(Bz*esdTrack.fP[4])*esdTrack.fIp.GetParameterAtRadius(120,5,10):esdTrack.fP[4]","esdTrack.fITSncls>4&&abs(esdTrack.fP[4])<0.25&&esdTrack.GetTPCClusterInfo(3,0)>0.75&&abs(esdTrack.fP[3])<0.8","colz",1000000)

In [ ]:
df= ROOT.RDataFrame(tree)

In [ ]:
ROOT.gInterpreter.ProcessLine("auto x4 = [&](int i) { std::cout << i << std::endl; };")

In [ ]:
ROOT.x4(1)

In [ ]:
histo=df.Histo1D("contSPD")
histo.Draw()

In [ ]:
c.Draw()